# 匿名函数
当我们传入函数的时候,有时候不需要显示地定义函数, 直接传入匿名函数更方便.

在Python中，对匿名函数提供了有限支持。还是以map()函数为例，计算f(x)=x2时，除了定义一个f(x)的函数外，还可以直接传入匿名函数：

In [2]:
list(map(lambda x:x*x,list(range(10))))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

关键字lambda表示匿名函数，冒号前面的x表示函数参数。

匿名函数有个限制，就是只能有一个表达式，不用写return，返回值就是该表达式的结果。

用匿名函数有个好处，因为函数没有名字，不必担心函数名冲突。此外，匿名函数也是一个函数对象，也可以把匿名函数赋值给一个变量，再利用变量来调用该函数：

In [7]:
f=lambda x:x*x
print(f(5))
# 也可以将匿名函数作为返回值返回
def build(x, y):
    return lambda: x * x + y * y
print(build(2,3)())
it=build(2,3)
print(it())

25
13
13


# 装饰器
由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数。

函数对象有一个__name__属性，可以拿到函数的名字：

In [13]:
def now():
    print('hello')
f=now
print(now.__name__)
print(f.__name__)

now
now


假设我们要增强now()函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。

本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下

In [25]:
def log(func):
    def wrapper(*args,**kw):
        print('call %s:' % func.__name__)
        return func(*args,**kw)
    return wrapper
@log
def now():
    print('2018-9-9')
now()

call now:
2018-9-9


上面的log是个decorator, 它接受一个函数作为参数, 并且返回一个函数. 我们需要借助python的@语法, 把decorator置于函数的定义处. 

调用now时, 不仅会运行now()本身, 还会在now()函数前打印一行日志

把@log 放在now()的定义处, 相当于执行了语句
 
now=log(now)

由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。

wrapper()函数的参数定义是(*args, **kw)，因此，wrapper()函数可以接受任意参数的调用。在wrapper()函数内，首先打印日志，再紧接着调用原始函数。

如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。

In [33]:
# 若要自定义log的文本
def log(text):
    def decorator(func): 
        def wrapper(*args,**kw):
            print('%s %s():'%(text,func.__name__))
            return func(*args,**kw)
        return wrapper
    return decorator
@log('excute')
def now():
    print('2018-9-9')
now()

excute now():
2018-9-9


执行效果相当于new=log('execute')(now)

首先执行log('execute')，返回的是decorator函数，再调用返回的函数，参数是now函数，返回值最终是wrapper函数。

以上两种decorator的定义都没有问题，但还差最后一步。因为我们讲了函数也是对象，它有__name__等属性，但你去看经过decorator装饰之后的函数，它们的__name__已经从原来的'now'变成了'wrapper'

因为返回的那个wrapper()函数名字就是'wrapper'，所以，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。

不需要编写wrapper.__name__ = func.__name__这样的代码，Python内置的functools.wraps就是干这个事的，所以，一个完整的decorator的写法如下

In [4]:
import functools
'''
def log(func):
    @functools.wraps(func)
    def wrapper(*args,**kw):
        print('call %s' % func.__name__)
        return func(*args,**kw)
    return wrapper
'''
def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args,**kw):
            print('%s %s()' % (text,func.__name__))
            return func(*args,**kw)
        return wrapper
    return decorator
@log('test')
def now():
    print('2018-9-9')
now()

test now()
2018-9-9


# 偏函数
通过设定参数的默认值，可以降低函数调用的难度。而偏函数也可以做到这一点。举例如下：

int()函数可以把字符串转换为整数，当仅传入字符串时，int()函数默认按十进制转换

In [43]:
print(int('12345'))
print(int('12345',base=8))  #以八进制进行转换为十进制
print(int('12345',16))      #以十六进制进行转换为十进制
print(int('1111011',base=2))#以二进制转换为十进制
print(bin(123))             #把数转换为二进制
print(oct(123))             #把数转换为八进制
print(hex(123))             #把数转化为十六进制

12345
5349
74565
123
0b1111011
0o173
0x7b


functools.partial就是帮助我们创建一个偏函数的，不需要我们自己定义int2()，可以直接使用下面的代码创建一个新的函数int2

所以，简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值

In [50]:
import functools
int2=functools.partial(int,base=2)
print(int2('1001001001'))
print(int2('123456',base=8))

585
42798


创建偏函数时, 实际可接收函数对象,args,kw,

当传入int2=functools.partial(int,base=2)时

实际固定了int()函数的关键字base 相当于

In [56]:
kw={'base':2}
print(int('1001',**kw))

max2=functools.partial(max,10) #实际会把10 作为*args的一部分自动加到左边,
#也就是max2(5,6,7)=max(5,6,7,10)
print(max2(5,6,7))

9
10


当函数的参数个数太多，需要简化时，使用functools.partial可以创建一个新的函数，这个新函数可以固定住原函数的部分参数，从而在调用时更简单。

# 模块
为了编写可维护的代码，我们把很多函数分组，分别放到不同的文件里，这样，每个文件包含的代码就相对较少，很多编程语言都采用这种组织代码的方式。在Python中，一个.py文件就称之为一个模块（Module）。

现在，假设我们的abc和xyz这两个模块名字与其他模块冲突了，于是我们可以通过包来组织模块，避免冲突。方法是选择一个顶层包名，比如mycompany，按照如下目录存放：

mycompany

├─ __init__.py

├─ abc.py

└─ xyz.py

引入了包以后，只要顶层的包名不与别人冲突，那所有模块都不会与别人冲突。现在，abc.py模块的名字就变成了mycompany.abc，类似的，xyz.py的模块名变成了mycompany.xyz。

请注意，每一个包目录下面都会有一个__init__.py的文件，这个文件是必须存在的，否则，Python就把这个目录当成普通目录，而不是一个包。__init__.py可以是空文件，也可以有Python代码，因为__init__.py本身就是一个模块，而它的模块名就是mycompany。

类似的，可以有多级目录，组成多级层次的包结构。比如如下的目录结构：

mycompany

 ├─ web

│  ├─ __init__.py

│  ├─ utils.py

│  └─ ww.py

├─ __init__.py

├─ abc.py

└─ xyz.py

文件ww.py的模块名就是mycompany.web.www，两个文件utils.py的模块名分别是mycompany.utils和mycompany.web.utils。



Python本身就内置了很多非常有用的模块，只要安装完毕，这些模块就可以立刻使用。
我们以内建的sys模块为例，编写一个hello的模块

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
' a test module '
__author__ = 'Michael Liao'
import sys
def test():
    args = sys.argv
    if len(args)==1:
        print('Hello, world!')
    elif len(args)==2:
        print('Hello, %s!' % args[1])
    else:
        print('Too many arguments!')
if __name__=='__main__':
    test()